In [8]:
import pymorphy2

from spacy.lang import ru
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = set(nltk.corpus.stopwords.words('russian'))
Unused_chars=[':',',','.','-','\n','?','!']
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()


def tokenize(file_text):
    tokens = word_tokenize(file_text)
    tokens = [i for i in tokens if (i not in stop)]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens


def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2]
    tokens = [get_lemma(token) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import csv
def csv_read(filename):
    with open('Data\\'+filename+'.csv') as csvfile:
        input_file1 = csv.DictReader(csvfile,delimiter=";")
        input_file=list(input_file1)
        result=[]
        for d in input_file:
            #if d['Суперслово/словосочетание']!='' and d['Значение']!='':
            toAdd=d
            if d['Фрагмент текста']!='':
                last=d['Фрагмент текста']
            else:
                toAdd['Фрагмент текста']=last
            if toAdd['Словосочетание']!='' and toAdd['Значение']!='':
                result.append(toAdd)
            else:
                break
        return result

In [4]:
import pymorphy2
Unused_chars=[':',',','.','-','\n','?','!']

#Логический разделитель
Logic_Sep=['PREP','CONJ','PRCL']
Logic_AdVerb=['PRTF','PRTS','GRND']
Logic_Adj=['ADJF','ADJS']
verb=['VERB','INFN']
Separators=Logic_Sep+Logic_AdVerb+verb+Logic_Adj

def logic_construct_extractor(string):
    Main_noun=''
    verbs=[]
    temp_string=''
    state=False
    Logical_structure=[]
    for word in string.split():
        f_word=word
        for char in Unused_chars:
            if char in f_word:
                f_word=f_word.replace(char,'')
        p=morph.parse(f_word)[0]
        if p.tag.POS in Separators and len(temp_string)>0 and not state:
            Logical_structure.append(temp_string)
            temp_string=f_word+' '
            state=True
        else:
            temp_string+=f_word+' '
            state=False
    Logical_structure.append(temp_string)
    return Logical_structure


In [10]:
expert_symbol=csv_read('Chapter16')
for sym in expert_symbol:
    print(sym)

OrderedDict([('№', ''), ('Фрагмент текста', 'Солнце уже снижалось над Лысой Горой, и была эта гора оцеплена двойным оцеплением.\nТа кавалерийская ала, что перерезала прокуратору путь около полудня, рысью вышла к Хевровским воротам города. Путь для нее уже был приготовлен. Пехотинцы каппадокийской когорты отдавили в стороны скопища людей, мулов и верблюдов, и ала, рыся и поднимая до неба белые столбы пыли, вышла на перекресток, где сходились две дороги: южная, ведущая в Вифлеем, и северо-западная – в Яффу. '), ('Словосочетание', 'Солнце уже снижалось'), ('Определение', 'символ'), ('Значение', 'закат (в т.ч. в переносном значении)'), ('', '')])
OrderedDict([('№', ''), ('Фрагмент текста', 'Солнце уже снижалось над Лысой Горой, и была эта гора оцеплена двойным оцеплением.\nТа кавалерийская ала, что перерезала прокуратору путь около полудня, рысью вышла к Хевровским воротам города. Путь для нее уже был приготовлен. Пехотинцы каппадокийской когорты отдавили в стороны скопища людей, мулов и в

OrderedDict([('№', ''), ('Фрагмент текста', 'Ведра пустели быстро, и кавалеристы из разных взводов по очереди отправлялись за водой в балку под горой, где в жидкой тени тощих тутовых деревьев доживал свои дни на этой дьявольской жаре мутноватый ручей.'), ('Словосочетание', 'в жидкой тени тощих тутовых деревьев'), ('Определение', 'символ'), ('Значение', 'бессилие природы'), ('', '')])
OrderedDict([('№', ''), ('Фрагмент текста', 'Ведра пустели быстро, и кавалеристы из разных взводов по очереди отправлялись за водой в балку под горой, где в жидкой тени тощих тутовых деревьев доживал свои дни на этой дьявольской жаре мутноватый ручей.'), ('Словосочетание', 'доживал свои дни на этой дьявольской жаре'), ('Определение', 'аллегория'), ('Значение', 'тяжкий конец'), ('', '')])
OrderedDict([('№', ''), ('Фрагмент текста', 'Ведра пустели быстро, и кавалеристы из разных взводов по очереди отправлялись за водой в балку под горой, где в жидкой тени тощих тутовых деревьев доживал свои дни на этой дьяво

OrderedDict([('№', ''), ('Фрагмент текста', 'Тот человек в капюшоне поместился недалеко от столбов на трехногом табурете и сидел в благодушной неподвижности, изредка, впрочем, от скуки прутиком расковыривая песок.'), ('Словосочетание', 'человек в капюшоне'), ('Определение', 'символ'), ('Значение', 'загадочность, скрытность, тайная сила'), ('', '')])
OrderedDict([('№', ''), ('Фрагмент текста', 'Тот человек в капюшоне поместился недалеко от столбов на трехногом табурете и сидел в благодушной неподвижности, изредка, впрочем, от скуки прутиком расковыривая песок.'), ('Словосочетание', 'в благодушной неподвижности'), ('Определение', 'символ'), ('Значение', 'довольство'), ('', '')])
OrderedDict([('№', ''), ('Фрагмент текста', 'Тот человек в капюшоне поместился недалеко от столбов на трехногом табурете и сидел в благодушной неподвижности, изредка, впрочем, от скуки прутиком расковыривая песок.'), ('Словосочетание', 'от скуки прутиком расковыривая песок'), ('Определение', 'символ'), ('Значение

OrderedDict([('№', ''), ('Фрагмент текста', 'Теперь, сидя на камне, этот чернобородый, с гноящимися от солнца и бессонницы глазами человек тосковал. Он то вздыхал, открывая свой истасканный в скитаниях, из голубого превратившийся в грязно-серый таллиф, и обнажал ушибленную копьем грудь, по которой стекал грязный пот, то в невыносимой муке поднимал глаза в небо, следя за тремя стервятниками, давно уже плававшими в вышине большими кругами в предчувствии скорого пира, то вперял безнадежный взор в желтую землю и видел на ней полуразрушенный собачий череп и бегающих вокруг него ящериц.'), ('Словосочетание', 'вперял безнадежный взор в желтую землю'), ('Определение', 'символ'), ('Значение', 'отчаяние'), ('', '')])
OrderedDict([('№', ''), ('Фрагмент текста', 'Теперь, сидя на камне, этот чернобородый, с гноящимися от солнца и бессонницы глазами человек тосковал. Он то вздыхал, открывая свой истасканный в скитаниях, из голубого превратившийся в грязно-серый таллиф, и обнажал ушибленную копьем гр

OrderedDict([('№', ''), ('Фрагмент текста', 'Тут что-то дунуло в лицо бывшему сборщику и что-то зашелестело у него под ногами. Дунуло еще раз, и тогда, открыв глаза, Левий увидел, что все в мире, под влиянием ли его проклятий или в силу каких-либо других причин, изменилось. Солнце исчезло, не дойдя до моря, в котором тонуло ежевечерне. Поглотив его, по небу с запада поднималась грозно и неуклонно грозовая туча. Края ее уже вскипали белой пеной, черное дымное брюхо отсвечивало желтым. Туча ворчала, и из нее время от времени вываливались огненные нити. По Яффской дороге, по скудной Гионской долине, над шатрами богомольцев, гонимые внезапно поднявшимся ветром, летели пыльные столбы. '), ('Словосочетание', 'Края ее уже вскипали белой пеной'), ('Определение', 'аллегория'), ('Значение', 'сила внезапности'), ('', '')])
OrderedDict([('№', ''), ('Фрагмент текста', 'Тут что-то дунуло в лицо бывшему сборщику и что-то зашелестело у него под ногами. Дунуло еще раз, и тогда, открыв глаза, Левий увид

In [11]:
import csv

with open('Chapter16_Only1.csv', mode='w') as output:
    count=0
    writer = csv.DictWriter(output, delimiter=';',fieldnames=['arg1','arg2','verb','sentence','verb_idx','label'])
    for sym in expert_symbol:
        count+=1
        writer.writerow([,,,sym['Фрагмент текста'],count,1])

TypeError: __init__() missing 1 required positional argument: 'fieldnames'